In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 5.0MB/s 


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import datetime
import os
import numpy as np

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
## ADDITIONAL FUNCTIONS
if not os.path.exists('models'):
    os.makedirs('models')
    
pStopWordsList = stopwords.words('english') + stopwords.words('russian')

def convert_amount(pAmount, rubles = False):
    try:
        if rubles:
            cAmount = pytils.numeral.rubles(abs(float(pAmount)), zero_for_kopeck=True)    
        else:
            cAmount = pytils.numeral.in_words(abs(int(pAmount)))    
    except Exception as e:
        cAmount = pAmount  
    return cAmount

def convert_date(pDate, format_mode, formatDate = None):
    formatDates = ['%Y-%m-%d',
                   '%d-%m-%Y',
                   '%d/%m/%Y',
                   '%d.%m.%Y',
                   '%y-%m-%d',
                   '%d-%m-%y',
                   '%d/%m/%y',
                   '%d.%m.%y',
                   '%Y-%m-%d %H:%M:%S',
                   '%d-%m-%Y %H:%M:%S',
                   '%d/%m/%Y %H:%M:%S',
                   '%d.%m.%Y %H:%M:%S',
                   '%y-%m-%d %H:%M:%S',
                   '%d-%m-%y %H:%M:%S',
                   '%d/%m/%y %H:%M:%S',
                   '%d.%m.%y %H:%M:%S']
    if formatDate!=None:
        cDate = datetime.datetime.strptime(pDate, formatDate).date()
    else:
        for fd in formatDates:
            try:
                cDate = datetime.datetime.strptime(pDate, fd).date()
                break
            except Exception as e:
                cDate=None
    if cDate == None:
        print('convert_date: Can not define date format for "%s"' % pDate)
        return ''
    if format_mode == 1:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year - 2000)
    if format_mode == 2:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '{} {} {}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)
    if format_mode == 3:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '"{}" {} {}'
        res = dformat.format(str(cDate.day).zfill(2), dmonth[cDate.month], cDate.year)            
    if format_mode == 4:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)            
    return res

def checkdate(text, pattern):
    res = ''
    if pattern==None: return ''
    text = text.lower()
    pattern = pattern.lower()
    res = pattern
    if res in text: return res
    res = convert_date(pattern, format_mode=1).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=4).lower()
    if res in text: return res   
    res = res.replace('.','/')
    if res in text: return res   
    res = res.replace('/','-')
    if res in text: return res       
    res = convert_date(pattern, format_mode=2).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=3).lower()
    if res in text: return res
    res = res[1:]
    if res in text: return res
    res = res.replace('""', '')
    if res in text: return res
    
    if " ".join([w for w in convert_date(pattern, format_mode=1).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=1)])
    if " ".join([w for w in convert_date(pattern, format_mode=2).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=2)])
    
    return ''
    
def checkamount(text, pattern, space =3):
    digits = ["0","1","2","3","4","5","6","7","8","9"]
    if pattern in (None,''): return ''
    res = ''
    
    if pattern in (None,'','nan'): return ''
    
    if pattern == 'nan':
        return res
    if pattern in text:
        res = pattern
    res=pattern
    if res in text: return res
    
    pattern = pattern.lower().replace(' ','').replace(',', '.')#.replace('-', '')
    res = pattern
    if res in text: return res
    
    res = pattern.replace('.', ',')
    if res in text: return res
    
    res = pattern.replace(',', '.')
    if res in text: return res

    res = pattern.replace(',', '-')
    if res in text: return res

    res = pattern.replace('.', '-')
    if res in text: return res
    
    try:
        res = '{:_}'.format(float(pattern)).replace('_', ' ')
        if res in text: return res
        res = res.replace('.',',')
        if res in text: return res
        
        res = res.replace('.','-')
        if res in text: return res
    except Exception as e:
        tmp = None
    
    res = convert_amount(pattern).lower()
    if convert_amount(pattern, rubles=True).lower() in text:
    
        res = convert_amount(pattern, rubles=True).lower()        
    if res in text: return res
        
    tmp = pattern.split('.')
    res = convert_amount(tmp[0]).lower()
    tmp[0] = '{:,}'.format(int(tmp[0])).replace(',', ' ')
    
    res = ','.join(tmp)
    if res in text: return res
    
    res = '.'.join(tmp)
    if res in text: return res
    
    reg = ''
    for l in pattern:
        if l in digits:
            reg = '%s%s.{0,%s}' % (reg, l,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]

    for w in pattern.split():
        reg = '%s%s.{0,%s}' % (reg, w,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]
    
    return ''

def checkstring(text, target, spaces = 2):
    if target in (None,'','nan'): return ''
    text = text.lower()
    target = target.lower()
    '''
    reg = ''
    for l in target:
        reg = '%s%s.{0,%s}' % (re.escape(reg), l,spaces)
    
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    '''
    target = target.replace('ооо','').replace('ао бывш.зао', '').\
             replace('зао','').replace('оао','').replace('ао','').\
             replace('ип','').replace(',','').strip()
    if target in text:
        return target
    if target in ['бн', 'б/н']:
        return ''
    if target.replace('0','o') in text.replace('0','o'):
        return target.replace('0','o')
    if target.replace('з','3') in text.replace('з','3'):
        return target.replace('з','3')    
    if target.replace('э','3') in text.replace('э','3'):
        return target.replace('э','3')    
    if target.replace('э','9') in text.replace('э','9'):
        return target.replace('э','9')    
    if target.replace('б','6') in text.replace('б','6'):
        return target.replace('б','6')    
    for ch in [' ','_', '\\','`','*','{','}','[',']','(',')','>','#','+','-','.','!','$','\'', ' ']:
        if ch in target:
            target = target.replace(ch,"")
            if target in text:
                return target
    return ''

def process_text_layer(TextLayer, pStopWordsList = []):
    txt =  " ".join([w for w in TextLayer.lower().split() \
                     if (not w in pStopWordsList)])

    txt = txt. \
        replace('«', ''). \
        replace('»', ''). \
        replace('(', ''). \
        replace(')', ''). \
        replace('\[)', ''). \
        replace('\]', ''). \
        replace('^', ''). \
        replace('\\', '')

    return txt

def tagging_sentence_by_targets(ptag, sentence_tagged, df_targets, pmodel_name):
    ret = sentence_tagged
    for r in df_targets[df_targets['tag']==ptag]['string_value']:
        tagging_sentence(ret, r, pmodel_name)
    
    return ret

def word2features2(sent, i, exclude = 'O', bias=3):
    word = sent[i][0]
    if sent[i][2] == exclude:
        return None

    features = {
        'bias': bias,
        'word.lower()': word.lower(),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'BOS': False,
        'EOS': False
    }
    for w in range(1, bias+1):
        if i > w:
            word1 = sent[i-w][0]
            features.update({
                '-%s:word.lower()' % (w): word1.lower()
            })
            
    if i < 1:
        features['BOS'] = True

    for w in range(1, bias+1):
        if i < len(sent)-w:
            word1 = sent[i+w][0]
            features.update({
                '+%s:word.lower()' % (w): word1.lower()
            })
            
    if i >= len(sent)-1:
        features['EOS'] = True

    return features

def sent2features(sent, exclude = 'O'):
    result = list()
    for i in range(len(sent)):
        ret = word2features2(sent, i, exclude = exclude, bias=10)
        if ret !=None:
            result.append(ret)
    return result

def sent2labels(sent, exclude = 'O'):
    result = list()
    for token, postag, label in sent:
        if label !=exclude:
            result.append(label)        
    return result

def tagging_sentence(sentence, target, tag):
    sentence_text=  ' '.join([w['Word'] for w in sentence])
    r_tag = ''

    try:
        res_find  = re.finditer(target.lower(), sentence_text.lower())
    except:
        return sentence

    for m in res_find:
        r_tag = 'B-%s' % tag
        for i in range(len(sentence_text[:m.start()].split()), len(sentence_text[:m.end()].split())):
            """
            if r_tag==('B-%s' % tag):
                if i>0:
                    sentence[i-1]['Tag']= 'B1-%s' % tag
                if i>1:
                    sentence[i-2]['Tag']= 'B2-%s' % tag                      
                if i>2:
                    sentence[i-3]['Tag']= 'B3-%s' % tag
                if i>3:
                    sentence[i-4]['Tag']= 'B4-%s' % tag  
            """
            sentence[i]['Tag']= r_tag
            r_tag = 'I-%s' % tag   
            """
            for j in range(1,5):
                if i+j < len(sentence):
                    sentence[i+j]['Tag']='E%s-%s' % (str(j), tag)
            """
    return sentence

def prepare_sentence(num, sentence):
    words = sentence.replace('"', '').split()
    tmp = []
    for w in words:
        tmp.append({'Sentence #': 'Sentence: %s' % num, 'Word' : w, 'POS' : 'NN', 'Tag': 'O'})
    return tmp

In [ ]:
!gdown --id 1hYI4ovDNu2jUUBOMAEKya6BPbYEBd5An

Downloading...
From: https://drive.google.com/uc?id=1hYI4ovDNu2jUUBOMAEKya6BPbYEBd5An
To: /content/NER PAD.csv
41.8MB [00:00, 158MB/s]


In [ ]:
data = pd.read_csv('NER PAD.csv')
for column in data.columns:
    data[column] = data[column].astype(str)
data['DOCCPTYINN'] = data['DOCCPTYINN'].apply(lambda r: r[:-2] if type(r) == str else r)

In [ ]:
PADData = data.copy()
PADData['string_value'] = PADData.apply(lambda r: process_text_layer(r['string_value'], pStopWordsList), axis = 1)

In [ ]:
PADData.head()

,tag,DOCTYPE,DOCNUM,DOCDATE,DOCAMOUNT,DOCCPTY,DOCAGRNUM,DOCAGRDATE,DOCCUSTOMER,DOCCPTYINN,string_value
0,SV990125172_ОРГ__INM_18'09'58_1.pdf,УПД2,(нацио,2018-12-31,1200,"Телеком ЛТД, ООО",П-1856у,2014-12-01,ТВ ДАРЬЯЛ,21190038,l * универсальный приложение № 1 постановлению...
1,SS528581_ОРГ__INM_17'05'39_A.pdf,АКТ,0000-000346,2019-04-10,10000,"РИА КАЛМЫКИЯ, АУ РК",0000-000290,2019-04-10,МИЦ ИЗВЕСТИЯ ООО,08160051,l лист согласования стр. 1 1 2 9 * 0 4 сохрани...
2,SF435712_ОРГ__INM_15'25'41_A.pdf,Счет-фактура,0000-004358,2019-03-31,"5273,76",ГОРМОСТ ГБУ,291-2013/ИК/А-158/13.,2013-04-01,АКЦЕПТ (ТЕЛЕВИЗИОННЫЙ КАНАЛ РЕН ТВ) ООО,77227654,l приложение № постановлению правительства рф ...
3,SV990124943_ОРГ__INM_19'00'43_A.pdf,АКТ,0000-012544,2019-06-30,1000,СТАТУС АО,Основной договор (Счет №А7407),2016-04-20,СПОРТ-ЭКСПРЕСС АО,77071792,"l акционерное общество ""регистраторское общест..."
4,S29013419_ОРИГИНАЛ__IND_1_22.05.12'21'36.pdf,АКТ,00000000014,2019-04-30,11800,СТАНДАРТ БЕЗОПАСНОСТИ ООО,04-14/ТРК-863/13СЖ,2013-12-13,Телерадиокомпания Петербург АО,78133587,"l * 000""стандарт безопасности"" адрес: 197376, ..."


In [ ]:
models = ['DOCNUM',
            'DOCDATE',
            'DOCAMOUNT',
            'DOCCPTY',
            'DOCAGRNUM',
            'DOCAGRDATE',
            'DOCCUSTOMER',
            'DOCCPTYINN']

datasets = {}
for model_name in models:
    ds = PADData[['tag','string_value', model_name]].rename(columns={'string_value': 'x', model_name: 'target'})
    if 'DATE' in model_name:
        ds['target'] = ds.apply(lambda r: checkdate(r['x'], r['target']),axis=1).copy()
    if model_name in ['DOCNUM', 'DOCAGRNUM', 'DOCCUSTOMER', 'DOCCPTY']:
        ds['target'] = ds.apply(lambda r: checkstring(r["x"], r['target']),axis=1).copy()      
    if 'AMOUNT' in model_name:
        ds['target'] = ds.apply(lambda r: checkamount(r['x'], r['target']),axis=1).copy()
        
    ds = ds[ds['target']!=''].drop_duplicates().copy()
    ds["x_tagged"] = ds.apply(lambda r: prepare_sentence(r.name, r["x"]), axis=1)
    ds["x_tagged"] = ds.apply(lambda r: tagging_sentence(r['x_tagged'], r['target'], model_name), axis=1)
    
    datasets[model_name] = ds.copy()

In [78]:
def choose(word, x, y):
    x = x.strip()
    y = y.strip()
    word = word.strip()
    nx = len(x)
    ny = len(y)
    n = len(word)

    find_x = -1
    find_y = -1
    if x.find(word) != -1:
        find_x = max(len(word), find_x)
    if word.find(x) != -1:
        find_x = max(len(x), find_x)
    if y.find(word) != -1:
        find_y = max(len(word), find_y)
    if word.find(y) != -1:
        find_y = max(len(y), find_y)

    if find_x != -1 or find_y != -1:
        if find_x >= find_y:
            return 0
        else:
            return 1
    
    index = 0
    while index < nx and index < ny and index < n:
        if x[index] != word[index] and y[index] == word[index]:
            return 1
        elif x[index] == word[index] and y[index] != word[index]:
             return 0
        index += 1

    if index < ny - 1 and index < n - 1 and y[index + 1] == word[index + 1]:
        return 1
    elif index < nx - 1 and index < n - 1 and x[index + 1] == word[index + 1]:
        return 0
    
    return 0 #default

def merge_tags(row):
    x = row.tags_x
    y = row.tags_y
    if type(y) != list:
        return
    if type(x) != list:
        x = y
        return
    assert len(x) == len(y)
    for i in range(len(x)):
        if x[i] == 'O' and y[i] != 'O':
            x[i] = y[i]
        if x[i] != 'O' and y[i] != 'O' and x[i] != y[i]: #labels conflict
            print(f'Labels conflict with word {row.words_x[i]} --- {x[i]} : {row.target_x}; {y[i]} : {row.target_y}')
            if pd.isnull(row.target_x):
                x[i] = y[i]
                continue
            if pd.isnull(row.target_y):
                continue
            choosed = choose(row.words_x[i], row.target_x, row.target_y)
            if choosed == 0:
                print(f'Selected : {row.target_x}')
            else:
                print(f'Selected : {row.target_y}')
                x[i] = y[i]

In [79]:
d_merged = datasets["DOCNUM"].copy()
d_merged["tags"] = d_merged.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
d_merged["words"] = d_merged.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
d_merged.head(1)

,tag,x,target,x_tagged,tags,words
2,SF435712_ОРГ__INM_15'25'41_A.pdf,l приложение № постановлению правительства рф ...,0000-004358,"[{'Sentence #': 'Sentence: 2', 'Word': 'l', 'P...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, O, O, O,...","[l, приложение, №, постановлению, правительств..."


In [80]:
for model in models[1:]:
    d_new = datasets[model].copy()
    d_new["tags"] = d_new.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
    d_new["words"] = d_new.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
    d_new.head(1)
    d_merged = pd.merge(d_merged, d_new.loc[:, ["tags", "target", "words", "tag"]], left_index=True, right_index=True, how="outer")
    d_merged.loc[pd.isna(d_merged.tags_x), 'tags_x'] = d_merged.loc[pd.isna(d_merged.tags_x), 'tags_y']
    d_merged.loc[pd.isna(d_merged.words_x), 'words_x'] = d_merged.loc[pd.isna(d_merged.words_x), 'words_y']
    d_merged.loc[pd.isna(d_merged.tag_x), 'tag_x'] = d_merged.loc[pd.isna(d_merged.tag_x), 'tag_y']
    d_merged.apply(merge_tags, axis=1)
    d_merged.drop(columns=['target_y', 'tags_y', 'words_y', 'tag_y'], inplace=True)
    d_merged.rename(columns={'target_x' : 'target', 'tags_x' : 'tags', 'words_x' : 'words', 'tag_x' : 'tag'}, inplace=True)

Labels conflict with word 16.04.2019 --- B-DOCNUM : 1; B-DOCDATE : 16.04.2019
Selected : 16.04.2019
Labels conflict with word 16.04.2019 --- B-DOCNUM : 1; B-DOCDATE : 16.04.2019
Selected : 16.04.2019
Labels conflict with word 10.08.2019 --- B-DOCNUM : 1; B-DOCDATE : 10.08.2019
Selected : 10.08.2019
Labels conflict with word 10.08.2019 --- B-DOCNUM : 1; B-DOCDATE : 10.08.2019
Selected : 10.08.2019
Labels conflict with word 19.03.2019 --- B-DOCNUM : 1; B-DOCDATE : 19.03.2019
Selected : 19.03.2019
Labels conflict with word 19.03.2019 --- B-DOCNUM : 1; B-DOCDATE : 19.03.2019
Selected : 19.03.2019
Labels conflict with word 18.04.2018 --- B-DOCNUM : 1; B-DOCDATE : 18.04.2018
Selected : 18.04.2018
Labels conflict with word 15 --- B-DOCNUM : 1; B-DOCDATE : 15 мая 2019
Selected : 15 мая 2019
Labels conflict with word 15 --- B-DOCNUM : 1; B-DOCDATE : 15 мая 2019
Selected : 15 мая 2019
Labels conflict with word 16 --- B-DOCNUM : 1; B-DOCDATE : 16 мая 2019
Selected : 16 мая 2019
Labels conflict wi

In [81]:
df = d_merged.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})

In [82]:
df.head()

,id,tokens,ner_tags
0,SV990125172_ОРГ__INM_18'09'58_1.pdf,"[l, *, универсальный, приложение, №, 1, постан...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-D..."
1,SS528581_ОРГ__INM_17'05'39_A.pdf,"[l, лист, согласования, стр., 1, 1, 2, 9, *, 0...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,SF435712_ОРГ__INM_15'25'41_A.pdf,"[l, приложение, №, постановлению, правительств...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, B-DOCDAT..."
3,SV990124943_ОРГ__INM_19'00'43_A.pdf,"[l, акционерное, общество, регистраторское, об...","[O, O, O, O, O, B-DOCCPTY, O, O, O, O, O, O, O..."
4,S29013419_ОРИГИНАЛ__IND_1_22.05.12'21'36.pdf,"[l, *, 000стандарт, безопасности, адрес:, 1973...","[O, O, O, B-DOCCPTY, O, O, O, O, O, O, O, O, O..."


In [83]:
df.shape

(4000, 3)

In [84]:
df.to_csv("NER_PAD_agg.csv", index=False)

In [86]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [87]:
train.head()

,id,tokens,ner_tags
3994,SV990115834_ОРГ__INM_16'04'09_3.pdf,"[l, испр, 4-, v, moscow, quarterly, report, li...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
423,B31001902_ОРГ__INM_15'54'06_5.pdf,"[l, приложение, №1, постановлению, правительст...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2991,A15279883_ОРИГИНАЛ__IND_2_31.05.15'26'21.pdf,"[l, лист, согласования, page, 1, 1, сохранить,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-DOCDATE..."
1221,S29013406_ОРИГИНАЛ__IND_1_22.05.11'50'49.pdf,"[l, приложение, ne, 1, постановлению, правител...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, B-DOCDAT..."
506,SV990126331_ОРГ__INM_16'23'42_A.pdf,"[l, универсальный, приложение, №, 1, постановл...","[O, O, O, O, O, O, O, O, O, O, O, O, B-DOCNUM,..."


In [88]:
test.head()

,id,tokens,ner_tags
555,B31005082_ОРГ__INM_15'39'47_1.pdf,"[l, т, !, приложение, №, 1, постановлению, пра...","[O, O, O, O, O, O, O, O, O, O, O, O, B-DOCNUM,..."
3491,SS821512_ОРГ__INM_15'39'55_A.pdf,"[l, 08.05.2019, лист, согласования, 15*0?, сох...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
527,B31001904_ОРГ__INM_15'54'06_5.pdf,"[l, приложение, №, 1, постановлению, правитель...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3925,SV990123251_ОРГ__INM_16'22'58_A.pdf,"[l, osa, e, p, r, n, n, k, v, v, 9, 9, 0, 1, а...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2989,SV990124219_16.07.2019.18'26'08.pdf,"[l, л, п, р, т, -, э, п, р, е, л, дрес, рассыл...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [89]:
print(train.shape, test.shape)

(3200, 3) (800, 3)


In [90]:
train.to_csv('NER_PAD_agg_train.csv', index=False)
test.to_csv('NER_PAD_agg_test.csv', index=False)

In [91]:
train_PAD, test_PAD = train_test_split(PADData, test_size=0.2, random_state=42)

In [93]:
set(train_PAD.index) == set(train.index)

True

In [97]:
set(test_PAD.index) == set(test.index)

True

In [98]:
train.to_csv('NER_PAD_train.csv', index=False)
test.to_csv('NER_PAD_test.csv', index=False)